In [2]:
import pandas as pd
import glob
import re
from datetime import datetime

In [9]:
def extract_from_json(file_to_process):
    if not re.search(r"\.json", file_to_process):
        raise TypeError("Incorrect file type passed")
    dataframe = pd.read_json(file_to_process)
    return dataframe

json_file_content_1 = extract_from_json("bank_market_cap_1.json")
print(json_file_content_1.shape)
json_file_content_1.head(10)

(70, 2)


,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
5,HSBC Holdings PLC,219.270
6,Agricultural Bank of China,203.244
7,Citigroup Inc.,203.165
8,Bank of China,181.469
9,China Merchants Bank,122.616


In [11]:
json_file_content_2 = extract_from_json("bank_market_cap_2.json")
print(json_file_content_2.shape)
json_file_content_2.head(10)

(35, 2)


,Name,Market Cap (US$ Billion)
35,Banco Bilbao Vizcaya Argentaria,60.678
36,Japan Post Bank,60.525
37,The Bank of New York Mellon,59.823
38,Shanghai Pudong Development Bank,58.796
39,Industrial Bank (China),57.388
40,Bank of China (Hong Kong),55.449
41,Bank of Montreal,53.181
42,Crédit Agricole,52.141
43,DBS Bank,51.475
44,Nordea,51.185


In [25]:
columns = ["Name", "Market Cap (US$ Billion)"]

def extract():
    extracted_data = pd.DataFrame(columns=columns)
    # for jsonfile in glob.glob("*.json"):
    #     extracted_data = extracted_data._append(extract_from_json(jsonfile), ignore_index=True)
        
    extracted_data = pd.concat([json_file_content_1, json_file_content_2], ignore_index=True, verify_integrity=True)
    return extracted_data

new_data_frame = extract()
print(new_data_frame.shape)
new_data_frame

(105, 2)


,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
100,Ping An Bank,37.993
101,Standard Chartered,37.319
102,United Overseas Bank,35.128
103,QNB Group,33.560


In [27]:
def read_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process, index_col=0)
    return dataframe

def extract_csv():
    for csv_file in glob.glob("*.csv"):
        extracted_csv_data = pd.DataFrame(read_csv(csv_file))
        
    return extracted_csv_data

csv_extracted_data = extract_csv()
csv_extracted_data.head(10)

,Rates
AUD,1.297088
BGN,1.608653
BRL,5.409196
CAD,1.271426
CHF,0.886083
CNY,6.483139
CZK,21.510117
DKK,6.119757
EUR,0.822504
GBP,0.732398


In [32]:
exchange_rate = float(csv_extracted_data.loc["GBP"])
print(exchange_rate)

def transform(data):
    data["Market Cap (US$ Billion)"] = round(data["Market Cap (US$ Billion)"]*exchange_rate, 3)
    data.rename(columns = {"Name": "Name", "Market Cap (US$ Billion)" : "Market Cap (GBP Billion)"}, inplace =True, errors="raised")
    
    return data
    
csv_transformed_data = transform(new_data_frame)
csv_transformed_data.head(5)

0.7323984208000001


C:\Users\sshivasw\AppData\Local\Temp\ipykernel_25620\2475231649.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  exchange_rate = float(csv_extracted_data.loc["GBP"])


,Name,Market Cap (GBP Billion)
0,JPMorgan Chase,209.700
1,Industrial and Commercial Bank of China,185.175
2,Bank of America,174.510
3,Wells Fargo,165.220
4,China Construction Bank,138.071


In [33]:
filename = "bank_market_cap_gbp.csv"

def load(filename, data):
    data.to_csv(filename, index=False)
    
logfilename = "bmc_log.txt"

def log(message):
    timestamp = datetime.now().strftime("Date: %h %d %Y - Time: %H:%M:%S")
    with open(logfilename, 'a') as FH:
        FH.write(f"{timestamp} -> {message}. \n")
        
log("ETL Job Started")
log("Running...")